In [1]:
import random
import pandas as pd
import os

from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

from PIL import Image, ImageFilter


import keras.backend as KerasBackend


import time
import numpy
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras import backend as K
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import UpSampling2D
from keras.layers import ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
import keras
import cv2

import tensorflow as tf
import keras.utils as np_utils

c:\users\buckf\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from IOU import iou_metric,iou_metric_batch,my_iou_metric

In [3]:
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [4]:
#This is the Data to be Trained and Labeled.
X = []
y = []

X_images = []
y_images = []

#Data Paths
train_image_path = r"C:\Users\buckf\Documents\Practicum_2\Data\train\images"
train_mask_path = r"C:\Users\buckf\Documents\Practicum_2\Data\train\masks"


image_list = os.listdir(train_image_path)
mask_list = os.listdir(train_mask_path)

for image_num in range(len(image_list)):
    #Gets the Path to the Mask
    certain_image_path = os.path.join(train_image_path, image_list[image_num])
    image_img = Image.open(certain_image_path).convert('L')
    image_pix = np.array(image_img.getdata()).reshape(image_img.size[0], image_img.size[1], 1)
    X.append(image_pix)
    X_images.append(image_img)

for mask_num in range(len(mask_list)):
    #Gets the Path to the Mask
    certain_mask_path = os.path.join(train_mask_path, mask_list[mask_num])
    mask_img = Image.open(certain_mask_path).convert('L')
    mask_pix = np.array(mask_img.getdata()).reshape(mask_img.size[0], mask_img.size[1], 1)
    y.append(mask_pix)
    y_images.append(mask_img)
    
X = np.array(X, dtype = 'uint8')
y = np.array(y, dtype = 'uint8')


#Convert pixel values between 0 and 1. 
X = np.divide(X, 255)
y = np.divide(y, 255)

#Covert Datatype of Arrays for openCV and IoU functions.
X = X.astype('uint8')
y = y.astype('uint8')

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size = 0.5, shuffle = True)

c:\users\buckf\anaconda3\envs\tensorflow\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
num_pixels = X_train.shape[1] * X_train.shape[2]
num_classes = y_val.shape[1]

def simple_cnn_model():
    
    # create model
### YOUR TURN
# Build a model that has 1 convolution layer, 1 max pooling, 1 dense, and output 
# Use 32 filters with 5x5 size
# For max pooling layer, make the layer such that the featuremap size would be the half after the pooling layer
# hint: you need to change the argument input_shape to (w,h,1) in the first conv layer

    
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size = (3,3),
                     activation = 'relu',
                     padding = 'same',
                     input_shape = (X_train.shape[1], X_train.shape[2], 1)))

    model.add(Conv2D(32, kernel_size = (3,3),
                     padding = 'same',
                     activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    
    model.add(Conv2DTranspose(filters=1, kernel_size=(3,3), strides=(2,2)))
    
    
    
    model.compile(loss= keras.losses.mean_squared_error, optimizer= 'adam' , metrics=[mean_iou])
    return model


    
model = simple_cnn_model()
model.summary()
#model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=15, verbose=1)


t0 = time.time()
log = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=6, batch_size=15, verbose=1)
t1 = time.time()
print(t1-t0, " seconds")
# Final evaluation of the model
scores = model.evaluate(X_val, y_val, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
print("Average MSE LOSS: " + str(scores[0]))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 101, 101, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 101, 101, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 101, 101, 1)       289       
Total params: 9,857
Trainable params: 9,857
Non-trainable params: 0
_________________________________________________________________
Train on 2000 samples, validate on 2000 samples
Epoch 1/6
1995/2000 [============================>.] - ETA: 0s - loss: 0.1915 - mean_iou: 0.

In [ ]:
num_pixels = X_train.shape[1] * X_train.shape[2]
num_classes = y_val.shape[1]

def simple_cnn_model():
    
    # create model
### YOUR TURN
# Build a model that has 1 convolution layer, 1 max pooling, 1 dense, and output 
# Use 32 filters with 5x5 size
# For max pooling layer, make the layer such that the featuremap size would be the half after the pooling layer
# hint: you need to change the argument input_shape to (w,h,1) in the first conv layer

    
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size = (3,3),
                     activation = 'relu',
                     padding = 'same',
                     input_shape = (X_train.shape[1], X_train.shape[2], 1)))

    model.add(Conv2D(32, kernel_size = (3,3),
                     padding = 'same',
                     activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    
    model.add(Conv2DTranspose(filters=1, kernel_size=(3,3), strides=(2,2)))
    
    
    
    model.compile(loss= keras.losses.mean_squared_error, optimizer= 'adam' , metrics=[my_iou_metric])
    return model


    
model = simple_cnn_model()
model.summary()
#model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=15, verbose=1)


t0 = time.time()
log = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=6, batch_size=15, verbose=1)
t1 = time.time()
print(t1-t0, " seconds")
# Final evaluation of the model
scores = model.evaluate(X_val, y_val, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
print("Average MSE LOSS: " + str(scores[0]))

In [ ]:
num_pixels = X_train.shape[1] * X_train.shape[2]
num_classes = y_val.shape[1]

def simple_cnn_model():
    
    # create model
### YOUR TURN
# Build a model that has 1 convolution layer, 1 max pooling, 1 dense, and output 
# Use 32 filters with 5x5 size
# For max pooling layer, make the layer such that the featuremap size would be the half after the pooling layer
# hint: you need to change the argument input_shape to (w,h,1) in the first conv layer

    
    
    model = Sequential()
    model.add(ZeroPadding2D(padding=(2,2), input_shape = (X_train.shape[1], X_train.shape[2], 1)))
    
    #Convolution 1:
    model.add(Conv2D(50, kernel_size = (3,3),
                     activation = 'relu',
                     padding = 'same'))
    
    #Convolution 2:
    model.add(Conv2D(50, kernel_size= (2,2), activation='relu', padding='same'))
    
    
    model.add(MaxPooling2D(pool_size=(3,3)))
    
    
    
    #Convolution 3
    model.add(Conv2D(50, kernel_size= (3,3), activation='relu', padding='same'))
    
    #Convolution 4
    model.add(Conv2D(50, kernel_size= (3,3), activation='relu', padding='same'))
    
    #Max-Pool 2
    model.add(MaxPooling2D(pool_size=(3,3)))
    
    
    model.add(Conv2DTranspose(1, kernel_size= (3,3), activation='relu', padding='same'))
    
    
    model.add(UpSampling2D(size=(3,3)))
    
    
    
    model.add(UpSampling2D(size=(3,3)))
    model.add(ZeroPadding2D(padding=(1,1)))
    
    
    
    model.compile(loss= keras.losses.mean_squared_error, optimizer= 'adam' , metrics=[my_iou_metric])
    return model
    

model = simple_cnn_model()
model.summary()
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=15, verbose=1)

In [ ]:
num_pixels = X_train.shape[1] * X_train.shape[2]
num_classes = y_val.shape[1]

def simple_cnn_model():
    
    # create model
### YOUR TURN
# Build a model that has 1 convolution layer, 1 max pooling, 1 dense, and output 
# Use 32 filters with 5x5 size
# For max pooling layer, make the layer such that the featuremap size would be the half after the pooling layer
# hint: you need to change the argument input_shape to (w,h,1) in the first conv layer

    
    
    model = Sequential()
    model.add(ZeroPadding2D(padding=(2,2), input_shape = (X_train.shape[1], X_train.shape[2], 1)))
    
    #Convolution 1:
    model.add(Conv2D(32, kernel_size = (3,3),
                     activation = 'relu',
                     padding = 'same'))
    
    #Convolution 2:
    model.add(Conv2D(64, kernel_size= (2,2), activation='relu', padding='same'))
    
    
    model.add(MaxPooling2D(pool_size=(3,3)))
    
    
    
    #Convolution 3
    model.add(Conv2D(128, kernel_size= (3,3), activation='relu', padding='same'))
    
    #Convolution 4
    model.add(Conv2D(264, kernel_size= (3,3), activation='relu', padding='same'))
    
    #Max-Pool 2
    model.add(MaxPooling2D(pool_size=(3,3)))
    
    
    model.add(Conv2DTranspose(264, kernel_size= (3,3), activation='relu', padding='same'))
    
    
    model.add(UpSampling2D(size=(3,3)))
    
    model.add(Conv2DTranspose(128, kernel_size= (3,3), activation='relu', padding='same'))
    
    model.add(UpSampling2D(size=(3,3)))
    
    model.add(Conv2DTranspose(1, kernel_size= (3,3), activation='relu', padding='same'))
    
    model.add(ZeroPadding2D(padding=(1,1)))
    
    
    
    model.compile(loss= keras.losses.mean_squared_error, optimizer= 'adam' , metrics=[my_iou_metric])
    return model
    

model = simple_cnn_model()
model.summary()
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=15, verbose=1)